# Tugas 2

## Distraksi
Distraksi merupakan transforasi kolom menjadi type data lain

In [ ]:
import pandas as pd
import numpy as np

In [ ]:
df = pd.read_csv("https://gist.githubusercontent.com/netj/8836201/raw/6f9306ad21398ea43cba4f7d537619d0e07d5ae3/iris.csv")
df